In [1]:
import numpy as np
import pandas as pd
import math
from tqdm.auto import tqdm

from scipy.sparse import linalg
from scipy.sparse import csr_matrix, csc_matrix
import scipy

import utils
import models

import time

In [2]:
data = pd.read_csv('data_train.csv')
train_data = utils.clean_df(data)
train_data.head()

,row,col,Prediction
0,43,0,4
1,60,0,3
2,66,0,4
3,71,0,3
4,85,0,5


## Naive implementation

In [3]:
def singular_value_thresholding(data, param, stepsize = 1.99):
    
    tao = param
    
    # Initialize
    Y = np.zeros([10000,1000])
    X = np.zeros([10000,1000])
    epsilon = 0.001
    err = 10
    
    
    A = np.zeros([10000,1000])
    row_ind = data.loc[:,'row']
    col_ind = data.loc[:,'col']
    A[row_ind, col_ind] = data.loc[:,'Prediction']
    

    while err >= epsilon:
        
        # Shrinkage operator
        u, s, vh = np.linalg.svd(Y, full_matrices = False)
        s_diag = np.diag(s) # Represent singular values in matrix
        s_new = s_diag - tao
        s_new[s_new < 0] = 0
        X = np.dot(u, np.dot(s_new, vh))
        
        # Projection
        proj = np.zeros([10000,1000])
        proj[row_ind, col_ind] = (A - X)[row_ind, col_ind]
        
        # Print error
        err = np.linalg.norm((X - A)[row_ind, col_ind], ord = 2)/np.linalg.norm(A[row_ind, col_ind])
        print("Error is: ", err)
        
        # Step forward
        Y = Y + stepsize * proj
    
    # Do final shrinkage
    u, s, vh = np.linalg.svd(Y)
    s_diag = np.concatenate((np.diag(s), np.zeros((9000,1000))), axis = 0) # Represent singular values in matrix
    s_new = s_diag - tao
    s_new[s_new < 0] = 0
    pred_matrix = np.dot(u, np.dot(s_new, vh))
    
    return pred_matrix

In [63]:
pred_matrix = singular_value_thresholding(train_data, param = 500, stepsize = 1)

Error is:  1.0
Error is:  6.559022766870573
Error is:  55.64959924674157
Error is:  507.18693044358474
Error is:  4659.458933518212
Error is:  42843.039169483905
Error is:  393972.6705687631
Error is:  3622900.3200248885
Error is:  33315563.180187136
Error is:  306364179.7621323
Error is:  2817272257.2368183
Error is:  25907150728.148342
Error is:  238237698622.0659
Error is:  2190792868010.5957
Error is:  20146154106990.75
Error is:  185260565354706.56
Error is:  1703624269588837.2


KeyboardInterrupt: 

## Now try cross validation

In [8]:
fold_1 = pd.read_csv('fold_1.csv', index_col = 0)
fold_2 = pd.read_csv('fold_2.csv', index_col = 0)
fold_3 = pd.read_csv('fold_3.csv', index_col = 0)
fold_4 = pd.read_csv('fold_4.csv', index_col = 0)
fold_5 = pd.read_csv('fold_5.csv', index_col = 0)
folds = [fold_1, fold_2, fold_3, fold_4, fold_5]

In [9]:
tao = np.array([500])
rmse = utils.k_fold_cv(folds, tao, singular_value_thresholding)

## too slow...

NameError: name 'singular_value_thresholding' is not defined

## Same implementation as in paper

In [31]:
# Implementation as in paper by Cai, Candes & Shen

def singular_value_thresholding2(data, param, stepsize = 1.99):
    
    tao = param
    
    # Initialize
    Y = np.zeros([10000,1000])
    X = np.zeros([10000,1000])
    epsilon = 0.1 ## Tune
    err = 10
    r = 1 # r=0 in paper
    l = 5
    
    A = np.zeros([10000,1000])
    row_ind = data.loc[:,'row']
    col_ind = data.loc[:,'col']
    A[row_ind, col_ind] = data.loc[:,'Prediction']
    
    

    while err >= epsilon:
        
        # Shrinkage operator
        sigma_min = tao + 1 # Make sure to enter while loop
        s = max(r, 1)  # s = r +1 in paper
        while sigma_min > tao:            
            u, sigma, vh = linalg.svds(csc_matrix(Y), k = min(s, 999)) # Compute SVD with s first singular values
            sigma_min = min(sigma) # look at this, was s[0] before. Should be the smallest singular value
            s = s + l
        r = np.count_nonzero(sigma > tao) # Update r to be the number of singular values > tao
        
        sigma_diag = np.diag(sigma) # Represent singular values in matrix
        sigma_new = sigma_diag - tao
        sigma_new[sigma_new < 0] = 0
        X = np.dot(u, np.dot(sigma_new, vh))
        
        # Projection
        proj = np.zeros([10000,1000])
        proj[row_ind, col_ind] = (A - X)[row_ind, col_ind]
        
        # Print error
        err = np.linalg.norm((X - A)[row_ind, col_ind], ord = 2)/np.linalg.norm(A[row_ind, col_ind])
        print("Error is: ", err)
        
        # Step forward
        Y = Y + stepsize * proj
    
    # Do final shrinkage
    u, s, vh = np.linalg.svd(Y)
    s_diag = np.concatenate((np.diag(s), np.zeros((9000,1000))), axis = 0) # Represent singular values in matrix
    s_new = s_diag - tao
    s_new[s_new < 0] = 0
    pred_matrix = np.dot(u, np.dot(s_new, vh))
    
    return pred_matrix

In [132]:
pred_matrix = singular_value_thresholding2(train_data, param = 10000, stepsize = 1.99)

Error is:  1.0
Error is:  1.0
Error is:  1.0
Error is:  0.7049881920503492
Error is:  0.5363885815842077
Error is:  0.48271987120531995
Error is:  0.44742632294615514
Error is:  0.4197623208120368
Error is:  0.39725733504870075
Error is:  0.3786496005068357
Error is:  0.3630839712567698
Error is:  0.34993582940492757
Error is:  0.3387354961752354
Error is:  0.3291231852021084
Error is:  0.32081906518951714
Error is:  0.31360255272246473
Error is:  0.307297670504084
Error is:  0.30064901964708973
Error is:  0.28978869835663773
Error is:  0.2855470612427575
Error is:  0.2824674559197408
Error is:  0.2798122500766365
Error is:  0.2774395770884058
Error is:  0.2752966468029135
Error is:  0.2733509824424347
Error is:  0.27157799772588187
Error is:  0.269957615004406
Error is:  0.2684729000061702
Error is:  0.26710932933242854
Error is:  0.2658543200562507
Error is:  0.2646968931753675
Error is:  0.2636274168241656
Error is:  0.2626374024081918
Error is:  0.2617193383647986
Error is:  0.2608

In [32]:
tao = np.array([10000])
rmse = utils.k_fold_cv(folds, tao, singular_value_thresholding2)

Error is:  1.0
Error is:  0.5592069853157406
Error is:  0.4629912824210239
Error is:  0.4716986057662303
Error is:  0.5049777144866365
Error is:  0.5644237513317066
Error is:  0.6110685118134049
Error is:  0.6791039925209449
Error is:  0.6988684536030433
Error is:  0.6976217153456769
Error is:  0.6902993455406875
Error is:  0.7220746854727621
Error is:  0.7412311562115403
Error is:  0.7479275584501005
Error is:  0.7827873337912072
Error is:  0.9474955017259393
Error is:  1.9277135379699974
Error is:  11.714828140389322
Error is:  102.2489785123168


KeyboardInterrupt: 

In [27]:
rmse

array([4.01632879])

In [133]:
# Try a submission

sample = pd.read_csv('sampleSubmission.csv')
temp = utils.clean_df(sample)

temp = utils.predict_scores(test_set = temp, pred_matrix = pred_matrix)
#temp.head()
sample['Prediction'] = temp['Prediction']
sample.to_csv("svt_tao_10000_2.csv", index = False)

## Fast randomized approximation
Paper by Oh et al

In [50]:
def gramschmidt(A):
    """
    Applies the Gram-Schmidt method to A
    and returns Q and R, so Q*R = A.
    """
    R = np.zeros((A.shape[1], A.shape[1]))
    Q = np.zeros(A.shape)
    for k in range(0, A.shape[1]):
        R[k, k] = np.sqrt(np.dot(A[:, k], A[:, k]))
        Q[:, k] = A[:, k]/R[k, k]
        for j in range(k+1, A.shape[1]):
            R[k, j] = np.dot(Q[:, k], A[:, j])
            A[:, j] = A[:, j] - R[k, j]*Q[:, k]
    return Q

In [58]:
def frsvt(A, tau, k, p, q, Q_tilda = 0): # q isn't used???
    m, n = A.shape
    l = k + p #>0
    if Q_tilda == 0:
        is_range_prop = False#
    # Find optimal (for NNM ~> lowest rank) X*
    # X* = SVT(A) ~=Q*SVT(B) to avoid SVD on A (mn)

    # Step 1: estimate orthonormal Q (mn) with k-rank << n
    if not is_range_prop:
        Sigma = np.random.multivariate_normal() #nl
        Y = A@Sigma #(ml)
        Q = sp.linalg.qr(Y, pivoting=True)# interface to lapack dgeqp3, also exists a low-level wrapper
    else:
        Sigma = np.random.multivariate_normal() # np
        Y = A@Sigma #Y(mp)
        ## Initialize Q via range propagation
            ## orthonormal biasis evolve slow
        Q_Y = gramschmidt(Y) #todo
        Q = concat_by_col(Q_tilda, Y)
    ## power interation
    eta = 2
    while eta > 0: #what is eta?
        Q = sp.linalg.qr(A@A.T@Q)
        eta -= 1
    #Step 2: SVT on B (~kn)
        ## SVD (by eigen decomposition if pos semi-def) + shrinkage on SV
    # C = WP = WVDV.T
    H, C = sp.linalg.qr(A.T@Q)
    # Form a pos semi-def, W(cpl, mn), P(cpl, nn, herm pos semi-def)
    W, P = sp.linalg.polar(C)
    V, D = sp.linalg.eig(P)
    Q_tilda = Q@V
    SS_D = np.sign(D) * np.max(np.abs(D)-tau, 0) #soft shrinkage
    SVT_A = Q_tilda@SS_D@(H@W@V).T
    return SVT_A, Q_tilda

In [59]:
frsvt(A = np.random.rand(3,3), Q_tilda = np.array([[1,0,0],[0,1,0],[0,0,1]]), tau = 2, k = 2, p = 1, q = 0)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## SVT without SVD (by Cai & Osher)

In [115]:
np.random.seed(0)
matrix = np.random.multivariate_normal(mean = np.zeros(500), cov = np.eye(500), size=500)
matrix.shape

(500, 500)

In [152]:
Z =np.reshape(np.arange(9), (3, 3))
Z@Z

array([[ 15,  18,  21],
       [ 42,  54,  66],
       [ 69,  90, 111]])

In [153]:
def svt_without_svd(Y, tau, delta):
    m, n = Y.shape
    epsilon = 1e-3
    max_iter = 1000
    
    # Step 1: compute polar decomposition Y = WZ
    W, Z = scipy.linalg.polar(Y)
    
    # Step 2: project Z onto 2-norm ball
    
    # Save computations
    Z_squared_25 = 0.25*Z@Z
    Z_25 = 0.25*Z
    Id = np.eye(n)
    I_75 = 0.75 * Id
    I_tau = tau*Id
    
    st1 = time.time()
    Sigma, V = scipy.linalg.eig(Z)
    Sigma1 = np.zeros(V.shape)
    Sigma1_vals = Sigma *(Sigma >= tau*(1-delta)) * (Sigma <= tau*(1+delta))
    np.fill_diagonal(Sigma1, Sigma1_vals)
    V1 = np.zeros(V.shape)
    idx = np.nonzero(Sigma1_vals)
    V1[:, idx] = V[:, idx]
    
    Z -= V1@Sigma1@V1.T
    error = epsilon * np.linalg.norm(Z, ord = 'fro')
    P = np.zeros((n,n))
    #print("eig decomp: {}".format(time.time()-st1))
    for i in np.arange(max_iter):
        #if i==0:
            #st2 = time.time()
        temp = 0.5*P + Z_25 + I_75 - np.linalg.inv(2*P - Z - I_tau)@(P - Z_squared_25 - I_75)
        if (np.linalg.norm(temp - P, ord = 'fro') <= error):
            proj_Sigma1 = np.zeros(Sigma1.shape)
            np.fill_diagonal(proj_Sigma1, tau)
            P = temp + V1@proj_Sigma1@V1.T #Need to fix
            break
        else:
            P = temp
        #if i==0:
            #print("one iteration: {}".format(time.time()-st2))
    proj_Z = P
    
    # Step 3: return SVT
    return Y - W@proj_Z
    

In [5]:
## Takes long and does not give correct results...

In [163]:
# Implementation as in paper by Cai, Candes & Shen

def singular_value_thresholding3(data, param, max_iter=100, stepsize = 0.5):
    
    tao = param
    
    # Initialize
    A = np.zeros([10000,1000])
    row_ind = data.loc[:,'row']
    col_ind = data.loc[:,'col']
    A[row_ind, col_ind] = data.loc[:,'Prediction']
    
    k0 = np.ceil(tao/(stepsize*np.linalg.norm(A, ord = 2)))
    Y = k0*stepsize*A
    X = np.zeros([10000,1000])
    epsilon = 1e-3 ## Tune
    err = 10
    r = 1 # r=0 in paper
    l = 5

    while max_iter > 0:
        
        # Shrinkage operator
        X = svt_without_svd(Y, tao, 0.01)
        
        # Projection
        proj = np.zeros([10000,1000])
        proj[row_ind, col_ind] = (A - X)[row_ind, col_ind]
        
        # Print error
        err = np.linalg.norm((X - A)[row_ind, col_ind])/np.linalg.norm(A[row_ind, col_ind])
        print("Error is: ", err)
        if err <= epsilon:
            break
        # Step forward
        Y = Y + stepsize * proj
        max_iter -= 1
    # Do final shrinkage
    u, s, vh = np.linalg.svd(X, full_matrices=False)
    s = s*(s > tao)
    print(vh.shape)
    pred_matrix = (u*s)@vh
    
    return np.clip(pred_matrix, 0, 5)
    #return X

In [147]:
pred_matrix = singular_value_thresholding3(train_data, param = 2000, max_iter=50, stepsize = 2)

/anaconda3/lib/python3.7/site-packages/numpy/lib/index_tricks.py:880: ComplexWarning: Casting complex values to real discards the imaginary part
  a.flat[:end:step] = val


Error is:  0.3296163401738494
Error is:  0.16971122497370475
Error is:  0.085708690686788
Error is:  0.04528649912517978
Error is:  0.023789794458969084
Error is:  0.012685338998507925
Error is:  0.006761880207732509
Error is:  0.0036264575293391216
Error is:  0.0019471981586994625
Error is:  0.001048676870486617
Error is:  0.0005655137375801376
(1000, 1000)


In [164]:
tao = np.array([250])
rmse = utils.k_fold_cv(folds, tao, singular_value_thresholding3)

Error is:  0.664319028156129
Error is:  0.3896916108234451
Error is:  0.2252090715079666
Error is:  0.14039398133918377
Error is:  0.09106214781938356
Error is:  0.07616104485730728
Error is:  0.06768857151973312
Error is:  0.07079460165366541
Error is:  0.058474480753967044
Error is:  0.031899190572141425
Error is:  0.01757453386023489
Error is:  0.05010146244916512
Error is:  0.06194175668866041
Error is:  0.04241444499337148
Error is:  0.04481840763957252
Error is:  0.04446204725336319
Error is:  0.04418049728064425
Error is:  0.04481565304962337
Error is:  0.05264863111452396
Error is:  0.03805093904059843
Error is:  0.05946513787284122
Error is:  0.040050402335787584
Error is:  0.04378710974064487
Error is:  0.043783202559875306
Error is:  0.044355010582203556
Error is:  0.043979445097907306
Error is:  0.05137489537470466
Error is:  0.04676903611187659
Error is:  0.03575357745495844
Error is:  0.05121084514296199
Error is:  0.038097999019091014
Error is:  0.05174197996328599
Error

/anaconda3/lib/python3.7/site-packages/numpy/lib/index_tricks.py:880: ComplexWarning: Casting complex values to real discards the imaginary part
  a.flat[:end:step] = val


Error is:  0.6642816898399101
Error is:  0.38964412874031606
Error is:  0.2207755656768084
Error is:  0.128690227624259
Error is:  0.09821915079612135
Error is:  0.07031085297603575
Error is:  0.0747293638880273
Error is:  0.06660624078708337
Error is:  0.05118021096835144
Error is:  0.027830743533299486
Error is:  0.031190904890695086
Error is:  0.0513811026249338
Error is:  0.05469604405330922
Error is:  0.039516454621161434
Error is:  0.043108193861643794
Error is:  0.022732686782636347
Error is:  0.06491123202679563
Error is:  0.03294140125154694
Error is:  0.05110945607834098
Error is:  0.03733079015337124
Error is:  0.05130507707542544
Error is:  0.03761383081740261
Error is:  0.05168367640526533
Error is:  0.02655395588637031
Error is:  0.0576944349033572
Error is:  0.028820658713034842
Error is:  0.058274997001645665
Error is:  0.029394330768106044
Error is:  0.05878646438930454
Error is:  0.02964481318989333
Error is:  0.056744318086781875
Error is:  0.029091121277017566
Error

/anaconda3/lib/python3.7/site-packages/numpy/lib/index_tricks.py:880: ComplexWarning: Casting complex values to real discards the imaginary part
  a.flat[:end:step] = val


Error is:  0.664309011795668
Error is:  0.3896776632467827
Error is:  0.22520138836566295
Error is:  0.13356004449362918
Error is:  0.09698870678488149
Error is:  0.07589829425698305
Error is:  0.06780085747624356
Error is:  0.058117957617260546
Error is:  0.030284629761573832
Error is:  0.03311249872955022
Error is:  0.0162599322723634
Error is:  0.048446423114619115
Error is:  0.02469659979305324
Error is:  0.05065906471699829
Error is:  0.0242037684018939
Error is:  0.050269215750599124
Error is:  0.03613390443059301
Error is:  0.033423356004191905
Error is:  0.04290398952489981
Error is:  0.021542956155265986
Error is:  0.049904556299039976
Error is:  0.024760282406774824
Error is:  0.0506532199378451
Error is:  0.024525624213322866
Error is:  0.05709635986974572
Error is:  0.03889225311931693
Error is:  0.034356918754698235
Error is:  0.04337664797488471
Error is:  0.02184310551192933
Error is:  0.06460740805224614
Error is:  0.032283860473195314
Error is:  0.03234172714996004
Err

/anaconda3/lib/python3.7/site-packages/numpy/lib/index_tricks.py:880: ComplexWarning: Casting complex values to real discards the imaginary part
  a.flat[:end:step] = val


Error is:  0.6643289079933526
Error is:  0.3897035509372396
Error is:  0.22519978851759093
Error is:  0.13376135769812042
Error is:  0.09151401903846818
Error is:  0.08251277452459967
Error is:  0.07541015278685494
Error is:  0.06695369573097217
Error is:  0.04305964784039019
Error is:  0.024561258411632418
Error is:  0.031080865704591294
Error is:  0.05797250792451602
Error is:  0.04082135833628441
Error is:  0.044823202641587304
Error is:  0.03547765321585971
Error is:  0.04404542000360972
Error is:  0.0534757972063275
Error is:  0.049225755332327925
Error is:  0.03748553704731316
Error is:  0.04282300451722839
Error is:  0.04428053209987319
Error is:  0.04482938399687479
Error is:  0.0460999183500798
Error is:  0.045434847303806224
Error is:  0.04476479458475715
Error is:  0.04437776769141234
Error is:  0.04590220025684662
Error is:  0.03653492296664564
Error is:  0.05202760524366872
Error is:  0.037856453399455064
Error is:  0.058955901942434684
Error is:  0.03040959623916692
Error

/anaconda3/lib/python3.7/site-packages/numpy/lib/index_tricks.py:880: ComplexWarning: Casting complex values to real discards the imaginary part
  a.flat[:end:step] = val


Error is:  0.6643174223464349
Error is:  0.3896907933606041
Error is:  0.22522040339158214
Error is:  0.1403656741364696
Error is:  0.09875380724396446
Error is:  0.08439971055500942
Error is:  0.07453509069638745
Error is:  0.060485440258402724
Error is:  0.03244022489530374
Error is:  0.01787550981821774
Error is:  0.009949917188879023
Error is:  0.07133426196710835
Error is:  0.03582916068023715
Error is:  0.0344555018450877
Error is:  0.017740685011461112
Error is:  0.05661754433362403
Error is:  0.028140329522121723
Error is:  0.059078226611483334
Error is:  0.029299197041630278
Error is:  0.040905691153770726
Error is:  0.02196560783141929
Error is:  0.05017736056056272
Error is:  0.02436587341056102
Error is:  0.06551790102210792
Error is:  0.03287145723294584
Error is:  0.032422557845655384
Error is:  0.031068869793915467
Error is:  0.04215126088995389
Error is:  0.03480258234067301
Error is:  0.05879079229755883
Error is:  0.029485022206756462
Error is:  0.04183237225634321
Er

In [165]:
rmse

array([3.0559276])

# Delete Below?

In [117]:
tau = np.sqrt(500)/2
delta = 0.5
m1 = svt_without_svd(matrix, tau, delta)

/anaconda3/lib/python3.7/site-packages/numpy/lib/index_tricks.py:880: ComplexWarning: Casting complex values to real discards the imaginary part
  a.flat[:end:step] = val


eig decomp: 0.10676002502441406
one iteration: 0.011145830154418945


In [119]:
np.linalg.matrix_rank(m1)

500

In [120]:
st = time.time()
np.linalg.svd(matrix, full_matrices = False)
print(time.time()-st)

0.047998905181884766


In [26]:
u, s, vh = np.linalg.svd(matrix, full_matrices = False)
s_diag = np.diag(s) # Represent singular values in matrix
s_new = s_diag - tau
s_new[s_new < 0] = 0
m2 = np.dot(u, np.dot(s_new, vh))

In [29]:
m2

array([[0.46834937, 0.47240482, 0.46675819, ..., 0.46871294, 0.46742548,
        0.46567567],
       [0.45623515, 0.4601857 , 0.45468512, ..., 0.45658931, 0.45533515,
        0.4536306 ],
       [0.47580777, 0.4799278 , 0.47419125, ..., 0.47617713, 0.47486917,
        0.47309149],
       ...,
       [0.47353118, 0.47763149, 0.47192239, ..., 0.47389876, 0.47259706,
        0.47082789],
       [0.46755861, 0.47160721, 0.46597012, ..., 0.46792156, 0.46663628,
        0.46488942],
       [0.44974799, 0.45364237, 0.44822001, ..., 0.45009711, 0.44886079,
        0.44718048]])